In [1]:
import pandas as pd
import os
import re
import numpy as np

print('Initialised!')

Initialised!


# Loading data and lookup / reference tables

In [7]:
path = './inputs/'

extensions = os.listdir(path)
paths = [path + '/' + e for e in extensions]

print("It looks like we're working with the following inputs:\n")
for p in paths:
  print('\t', p)

quadrat_path = './resources/quadrat_properties.csv'
species_path = './resources/life_form_matching.csv'

It looks like we're working with the following inputs:

	 ./inputs//Ecothining_understoreydata2020.xlsx


In [9]:
df = pd.read_excel(paths[0])

print('Loaded input data!')

quadrats = pd.read_csv(quadrat_path)
life_forms = pd.read_csv(species_path, index_col=0)

print('Loaded lookup tables!')

Loaded input data!
Loaded lookup tables!


# Cleaning input data, resolving conflicts

In [10]:
df = df[['Record_ID', 'Year_monitoring', 'Plot_number', 'Quadrat_number', 
         'T002_Flora_Species_name', 'Score']]
df.columns = ['Record ID', 'Year', 'Plot', 'Quadrat', 
         'Species Name', 'Score']
df = df.drop_duplicates()
df.index = range(0, len(df))

for i in quadrats.index:
  p = quadrats.loc[i, 'Plot Number']
  q = quadrats.loc[i, 'Quadrat Number']

  rows = df.loc[(df['Plot']==p) & (df['Quadrat']==q)].index

  df.at[rows, 'Treatment'] = quadrats.loc[i, 'Treatment']
  df.at[rows, 'Fenced'] = quadrats.loc[i, 'Fenced']
  df.at[rows, 'Gap'] = quadrats.loc[i, 'Gap']

## Let's take a look at a few rows:

In [11]:
df.head()

,Record ID,Year,Plot,Quadrat,Species Name,Score,Treatment,Fenced,Gap
0,10733,6,1,1,Stellaria spp.,0.5,Gap,False,False
1,10730,6,1,1,Rytidosperma spp.,1.0,Gap,False,False
2,10731,6,1,1,Rytidosperma spp.,0.5,Gap,False,False
3,10732,6,1,1,Senecio spp,0.5,Gap,False,False
4,10734,6,1,1,Wahlenbergia spp,0.5,Gap,False,False


# Life Form Matching

In [14]:
species = pd.unique(df['Species Name'])

for s in species:
  if s not in life_forms.index:
    life_forms.loc[s, :] = None

life_forms.to_csv(species_path)

not_matched = list(life_forms[life_forms['Life Form'].isna()].index)

print("The following species names are not currently matched to a life form:\n")
for i in not_matched:
  print(i)

The following species names are not currently matched to a life form:

CWD
Leaf Litter
Moss/Lichen
Cardamine sp.
Bare Earth
Cerastium sp.
Unknown Species
Eucalyptus tereteticornis
Dead Standing Timber
Standing dead timber
MoSs/Lichen
Asteraceae


# Apply matching and delete any un-matched records

In [15]:
life_forms = life_forms.loc[~life_forms['Life Form'].isna()]

for s in life_forms.index:
  df.loc[df['Species Name']==s, ['Life Form', 'Shade Tolerant', 'Weed']] = life_forms.loc[s].values

df = df.loc[~df['Life Form'].isna()]


SCORES = {0.5: 0.5, 1: 3, 2: 7.5, 3: 15,
          4: 25, 5: 35, 6: 45, 7: 55,
          8: 65, 9: 75, 10: 95}

for i in df.index:
  df.loc[i, 'Abundance'] = SCORES[df.loc[i, 'Score']]

In [16]:
output_path = './outputs/'
df.to_csv(output_path+'understorey_cleaned.csv', index=False)

print('Saved the cleaned and matched dataset!')

Saved the cleaned and matched dataset!


# Create Contingency Tables

In [14]:
df = df.loc[df['Weed']==False]
df = df.drop('Weed', axis=1)

In [ ]:
def Shannons (vector):

  p = [i/100 for i in vector]
  p = [i/sum(p) for i in p]
  p = [np.log(i)*i for i in p]
    
  return(-sum(p))
  

years = sorted(pd.unique(df['Year']))

df_aggregate = df.drop(['Record ID', 'Score'], axis=1)

df_aggregate = df_aggregate.groupby(['Treatment', 'Plot','Quadrat','Fenced',
                                     'Gap','Year','Life Form','Species Name']).sum()

df_aggregate_plot = df_aggregate.groupby(['Treatment', 'Plot','Fenced',
                                          'Gap','Year','Life Form','Species Name']).sum()

df_aggregate_treatment = df_aggregate.groupby(['Treatment','Fenced',
                                               'Gap','Year','Life Form','Species Name']).sum()

# RICHNESS & DIVERSITY
## Quadrat Level

In [ ]:
rows = df_aggregate.index.droplevel(df_aggregate.index.names[-3:])
rows = rows.drop_duplicates()

matrix_species_richness = pd.DataFrame(None, index=rows, columns=years)
matrix_species_diverstiy = pd.DataFrame(None, index=rows, columns=years)

to_calculate = df_aggregate.index.droplevel(['Life Form', 'Species Name']).drop_duplicates()

for i in to_calculate:

  vector = df_aggregate.loc[i, 'Abundance']

  # Species Richness
  matrix_species_richness.loc[i[:-1], i[-1:]] = len(pd.unique(vector.index))

  # Species Diversity
  matrix_species_diverstiy.loc[i[:-1], i[-1:]] = Shannons(vector)

In [ ]:
matrix_species_richness.to_csv(output_path+'richness_quadrat.csv')
matrix_species_diverstiy.to_csv(output_path+'diversity_quadrat.csv')

# RICHNESS | DIVERSITY
## Plot Level

In [ ]:
rows = df_aggregate_plot.index.droplevel(df_aggregate_plot.index.names[-3:])
rows = rows.drop_duplicates()

matrix_species_richness = pd.DataFrame(None, index=rows, columns=years)
matrix_species_diverstiy = pd.DataFrame(None, index=rows, columns=years)

to_calculate = df_aggregate_plot.index.droplevel(['Life Form', 'Species Name']).drop_duplicates()

for i in to_calculate:
  vector = df_aggregate_plot.loc[i, 'Abundance']
  
  # Species Richness
  matrix_species_richness.loc[i[:-1], i[-1:]] = len(pd.unique(vector.index))

  # Species Diversity
  matrix_species_diverstiy.loc[i[:-1], i[-1:]] = Shannons(vector)

In [ ]:
matrix_species_richness.to_csv(output_path+'richness_plot.csv')
matrix_species_diverstiy.to_csv(output_path+'diversity_plot.csv')

# RICHNESS | DIVERSITY
## Treatment Level

In [ ]:
rows = df_aggregate_treatment.index.droplevel(df_aggregate_treatment.index.names[-3:])
rows = rows.drop_duplicates()

matrix_species_richness = pd.DataFrame(None, index=rows, columns=years)
matrix_species_diverstiy = pd.DataFrame(None, index=rows, columns=years)

to_calculate = df_aggregate_treatment.index.droplevel(['Life Form', 'Species Name']).drop_duplicates()

for i in to_calculate:

  vector = df_aggregate_treatment.loc[i, 'Abundance']
  
  # Species Richness
  matrix_species_richness.loc[i[:-1], i[-1:]] = len(pd.unique(vector.index))

  # Species Diversity
  matrix_species_diverstiy.loc[i[:-1], i[-1:]] = Shannons(vector)

In [ ]:
matrix_species_richness.to_csv(output_path+'richness_treatment.csv')
matrix_species_diverstiy.to_csv(output_path+'diversity_treatment.csv')

# ABUNDANCE

In [ ]:
lf_abund = df_aggregate.groupby(['Treatment','Plot','Quadrat','Fenced',
                                 'Gap','Life Form','Year']).sum()

lf_abund_plot = lf_abund.groupby(['Treatment','Plot','Fenced',
                                  'Gap','Life Form','Year']).mean()

lf_abund_treatment = lf_abund.groupby(['Treatment','Fenced',
                                 'Gap','Life Form','Year']).mean()

In [ ]:
lf_abund.to_csv(output_path+'lf_abund_quadrat.csv')

In [ ]:
rows = lf_abund_plot.index.droplevel(lf_abund_plot.index.names[-2:])
rows = rows.drop_duplicates()

cols = lf_abund_plot.index.droplevel(lf_abund_plot.index.names[:-2])
cols = cols.drop_duplicates()

matrix_lf_abund = pd.DataFrame(index=rows, columns=cols)

for i in lf_abund_plot.index:
    matrix_lf_abund.at[i[:-2],i[-2:]] = lf_abund_plot.loc[i, 'Abundance']

matrix_lf_abund.fillna(0, inplace=True)
matrix_lf_abund.sort_index(axis=1, inplace=True)

In [ ]:
matrix_lf_abund.to_csv(output_path+'lf_abund_plot.csv')

In [ ]:
rows = lf_abund_treatment.index.droplevel(lf_abund_treatment.index.names[-2:])
rows = rows.drop_duplicates()

cols = lf_abund_treatment.index.droplevel(lf_abund_treatment.index.names[:-2])
cols = cols.drop_duplicates()

matrix_lf_abund = pd.DataFrame(index=rows, columns=cols)

for i in lf_abund_treatment.index:
    matrix_lf_abund.at[i[:-2],i[-2:]] = lf_abund_treatment.loc[i, 'Abundance']

matrix_lf_abund.fillna(0, inplace=True)
matrix_lf_abund.sort_index(axis=1, inplace=True)

In [ ]:
matrix_lf_abund.to_csv(output_path+'lf_abund_treatment.csv')